In [65]:
import requests
import pandas as pd

# Base URL for ASA API
BASE_URL = "https://app.americansocceranalysis.com/api/v1"

# List of endpoints
endpoints = [
    "mls/players",
    "mls/players/xgoals",
    "mls/players/xpass",
    "mls/players/goals-added",
    "mls/players/salaries",
    "mls/goalkeepers/xgoals",
    "mls/goalkeepers/goals-added",
    "mls/teams",
    "mls/teams/xgoals",
    "mls/teams/xpass",
    "mls/teams/goals-added",
    "mls/teams/salaries",
    "mls/games",
    "mls/games/xgoals",
    "mls/managers",
    "mls/referees",
    "mls/stadia"
]

# Function to fetch and display key fields
def fetch_key_fields(endpoint):
    url = f"{BASE_URL}/{endpoint}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses
        data = response.json()
        
        # Flatten JSON to get column names (if it's a list of dicts)
        if isinstance(data, list) and len(data) > 0:
            key_fields = pd.json_normalize(data).columns.tolist()
            print(f"=== Endpoint: {endpoint} ===")
            print(key_fields)
            return key_fields
        else:
            print(f"Endpoint {endpoint} returned unexpected data format.")
            return []
    except Exception as e:
        print(f"Error fetching {endpoint}: {e}")
        return []

# Iterate over endpoints and fetch key fields
endpoint_key_fields = {}
for endpoint in endpoints:
    key_fields = fetch_key_fields(endpoint)
    endpoint_key_fields[endpoint] = key_fields

# Display collected key fields for all endpoints
endpoint_key_fields_df = pd.DataFrame.from_dict(endpoint_key_fields, orient="index").T
endpoint_key_fields_df.fillna("", inplace=True)
endpoint_key_fields_df

=== Endpoint: mls/players ===
['player_id', 'player_name', 'birth_date', 'height_ft', 'height_in', 'weight_lb', 'nationality', 'primary_broad_position', 'primary_general_position', 'season_name', 'secondary_general_position', 'secondary_broad_position']
=== Endpoint: mls/players/xgoals ===
['player_id', 'team_id', 'general_position', 'minutes_played', 'shots', 'shots_on_target', 'goals', 'xgoals', 'xplace', 'goals_minus_xgoals', 'key_passes', 'primary_assists', 'xassists', 'primary_assists_minus_xassists', 'xgoals_plus_xassists', 'points_added', 'xpoints_added']
=== Endpoint: mls/players/xpass ===
['player_id', 'team_id', 'general_position', 'minutes_played', 'attempted_passes', 'pass_completion_percentage', 'xpass_completion_percentage', 'passes_completed_over_expected', 'passes_completed_over_expected_p100', 'avg_distance_yds', 'avg_vertical_distance_yds', 'share_team_touches', 'count_games']
=== Endpoint: mls/players/goals-added ===
['player_id', 'team_id', 'general_position', 'minu

,mls/players,mls/players/xgoals,mls/players/xpass,mls/players/goals-added,mls/players/salaries,mls/goalkeepers/xgoals,mls/goalkeepers/goals-added,mls/teams,mls/teams/xgoals,mls/teams/xpass,mls/teams/goals-added,mls/teams/salaries,mls/games,mls/games/xgoals,mls/managers,mls/referees,mls/stadia
0,player_id,player_id,player_id,player_id,player_id,player_id,player_id,team_id,team_id,team_id,team_id,,game_id,game_id,manager_id,referee_id,stadium_id
1,player_name,team_id,team_id,team_id,team_id,team_id,team_id,team_name,count_games,count_games,minutes,,date_time_utc,date_time_utc,manager_name,referee_name,stadium_name
2,birth_date,general_position,general_position,general_position,season_name,minutes_played,minutes_played,team_short_name,shots_for,attempted_passes_for,data,,home_score,home_team_id,nationality,birth_date,capacity
3,height_ft,minutes_played,minutes_played,minutes_played,position,shots_faced,data,team_abbreviation,shots_against,pass_completion_percentage_for,,,away_score,home_goals,,nationality,year_built
4,height_in,shots,attempted_passes,data,base_salary,goals_conceded,,,goals_for,xpass_completion_percentage_for,,,home_team_id,home_team_xgoals,,,roof
5,weight_lb,shots_on_target,pass_completion_percentage,,guaranteed_compensation,saves,,,goals_against,passes_completed_over_expected_for,,,away_team_id,home_player_xgoals,,,turf
6,nationality,goals,xpass_completion_percentage,,mlspa_release,share_headed_shots,,,goal_difference,passes_completed_over_expected_p100_for,,,referee_id,away_team_id,,,street
7,primary_broad_position,xgoals,passes_completed_over_expected,,,xgoals_gk_faced,,,xgoals_for,avg_vertical_distance_for,,,stadium_id,away_goals,,,city
8,primary_general_position,xplace,passes_completed_over_expected_p100,,,goals_minus_xgoals_gk,,,xgoals_against,attempted_passes_against,,,home_manager_id,away_team_xgoals,,,province
9,season_name,goals_minus_xgoals,avg_distance_yds,,,goals_divided_by_xgoals_gk,,,xgoal_difference,pass_completion_percentage_against,,,away_manager_id,away_player_xgoals,,,country


In [66]:
# Define endpoints and required columns
endpoints_and_columns = {
    "mls/players": [
        "player_id", "player_name", "birth_date", "primary_broad_position", "primary_general_position"
    ],
    "mls/players/xgoals": [
        "player_id", "team_id", "minutes_played", "shots", "shots_on_target",
        "goals", "xgoals", "xplace", "goals_minus_xgoals", "key_passes",
        "primary_assists", "xassists", "primary_assists_minus_xassists",
        "xgoals_plus_xassists", "points_added", "xpoints_added"
    ],
    "mls/players/xpass": [
        "player_id", "team_id", "minutes_played", "attempted_passes",
        "pass_completion_percentage", "xpass_completion_percentage",
        "passes_completed_over_expected", "avg_distance_yds"
    ],
    "mls/players/salaries": [
        "player_id", "team_id", "season_name", "position",
        "base_salary", "guaranteed_compensation"
    ],
    "mls/teams": [
        "team_id", "team_name"
    ],
    "mls/games": [
        "game_id", "date_time_utc", "home_score", "away_score",
        "home_team_id", "away_team_id"
    ]
}

# Function to fetch data from the API and filter columns
def fetch_data(endpoint, required_columns):
    url = f"{BASE_URL}/{endpoint}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        # Convert to DataFrame and filter for required columns
        if isinstance(data, list) and len(data) > 0:
            df = pd.json_normalize(data)
            filtered_df = df[required_columns]
            return filtered_df
        else:
            print(f"Endpoint {endpoint} returned unexpected data format.")
            return pd.DataFrame(columns=required_columns)
    except Exception as e:
        print(f"Error fetching {endpoint}: {e}")
        return pd.DataFrame(columns=required_columns)

# Initialize dictionary to store DataFrames for each endpoint
dataframes = {}

# Fetch data for each endpoint and store in the dictionary
for endpoint, columns in endpoints_and_columns.items():
    print(f"Fetching data from {endpoint}...")
    dataframes[endpoint] = fetch_data(endpoint, columns)

# Example: Access specific DataFrame
players_df = dataframes["mls/players"]
xgoals_df = dataframes["mls/players/xgoals"]

# Display or process data further
print("Players DataFrame preview:")
print(players_df.head())
print("\nXGoals DataFrame preview:")
print(xgoals_df.head())

# Save to CSV or proceed to SQL upload if needed
# players_df.to_csv("mls_players.csv", index=False)
# xgoals_df.to_csv("mls_xgoals.csv", index=False)


Fetching data from mls/players...
Fetching data from mls/players/xgoals...
Fetching data from mls/players/xpass...
Fetching data from mls/players/salaries...
Fetching data from mls/teams...
Fetching data from mls/games...
Players DataFrame preview:
    player_id          player_name  birth_date primary_broad_position  \
0  0Oq6006M6D          Ugo Ihemelu  1983-04-03                    NaN   
1  0Oq60APM6D      Jason Hernandez  1983-08-26                     DF   
2  0Oq60VgM6D     Jeff Larentowicz  1983-08-05                     MF   
3  0Oq624oPq6  Kalani Kossa-Rienzi  2002-06-27                     DF   
4  0Oq62GYAq6        Oscar Herrera  2002-03-28                    NaN   

  primary_general_position  
0                      NaN  
1                       CB  
2                       DM  
3                       FB  
4                      NaN  

XGoals DataFrame preview:
    player_id                               team_id  minutes_played  shots  \
0  0Oq60APM6D  [kRQabn8MKZ, 0KPqj

In [67]:
# Define a dictionary to log changes
change_logs = {}

# Function to log changes
def log_change(dataset_name, description):
    if dataset_name not in change_logs:
        change_logs[dataset_name] = []
    change_logs[dataset_name].append(f"{pd.Timestamp.now()} - {description}")

# Function to clean and validate a dataset
def clean_and_validate_dataset(df, dataset_name, dataframes):
    if df is None:
        return None  # Skip if DataFrame is missing

    # 1. Handle Missing Values
    if dataset_name == "mls/players":
        if "birth_date" in df.columns:
            df["birth_date"] = df["birth_date"].fillna("1900-01-01")
            df["birth_date"] = pd.to_datetime(df["birth_date"], errors="coerce")
            log_change(dataset_name, "Filled missing birth_date with '1900-01-01'.")
        for col in ["primary_broad_position", "primary_general_position"]:
            if col in df.columns:
                df[col] = df[col].fillna("UNKNOWN")
                log_change(dataset_name, f"Filled missing values in {col} with 'UNKNOWN'.")
    if dataset_name == "mls/players/salaries":
        if "team_id" in df.columns:
            missing_team_id_count = df["team_id"].isnull().sum()
            df = df.dropna(subset=["team_id"])
            log_change(dataset_name, f"Dropped {missing_team_id_count} rows with missing 'team_id'.")

    # 2. Handle Duplicates
    problematic_rows = df.applymap(lambda x: isinstance(x, (dict, list))).any(axis=1)
    if problematic_rows.any():
        count = problematic_rows.sum()
        df = df[~problematic_rows]
        log_change(dataset_name, f"Dropped {count} rows with unhashable types.")
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        df = df.drop_duplicates()
        log_change(dataset_name, f"Dropped {duplicate_count} duplicate rows.")
    duplicate_columns = df.columns[df.columns.duplicated()].tolist()
    if duplicate_columns:
        df = df.loc[:, ~df.columns.duplicated()]
        log_change(dataset_name, f"Dropped duplicate columns: {duplicate_columns}.")

    # 3. Validate Data Types and Cast for SQL Compatibility
    for col in df.columns:
        if col in ["birth_date", "date_time_utc"]:
            df[col] = pd.to_datetime(df[col], errors="coerce")
        elif pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].apply(lambda x: round(x, 6) if pd.notnull(x) and pd.api.types.is_number(x) else None)
        elif pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].apply(lambda x: str(x) if isinstance(x, (str, int, float)) else None)
        elif pd.api.types.is_integer_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast="integer", errors="coerce")

    # 4. Drop Unwanted Columns
    unwanted_columns = ["extra_time", "penalties", "home_penalties", "away_penalties"]
    unwanted_columns += [col for col in df.columns if "outlier" in col]
    dropped_columns = [col for col in unwanted_columns if col in df.columns]
    if dropped_columns:
        df = df.drop(columns=dropped_columns)
        log_change(dataset_name, f"Dropped unwanted columns: {dropped_columns}.")

    # 5. Split season_name Column if Applicable
    if "season_name" in df.columns:
        df["season_name"] = df["season_name"].apply(
            lambda x: x if isinstance(x, list) else [int(x)] if pd.notnull(x) else []
        )
        df["season_name_initial"] = df["season_name"].apply(lambda x: min(x) if x else None)
        df["season_name_current"] = df["season_name"].apply(lambda x: max(x) if x else None)
        df = df.drop(columns=["season_name"])
        log_change(dataset_name, "Split 'season_name' into 'season_name_initial' and 'season_name_current'.")

    # 6. Foreign Key Validation
    if dataset_name == "mls/players/xgoals" or dataset_name == "mls/players/xpass":
        player_ids = set(dataframes["mls/players"]["player_id"])
        df = df[df["player_id"].isin(player_ids)]
        log_change(dataset_name, "Validated and kept only valid player_id entries.")

    if dataset_name == "mls/games":
        team_ids = set(dataframes["mls/teams"]["team_id"])
        df = df[df["home_team_id"].isin(team_ids) & df["away_team_id"].isin(team_ids)]
        log_change(dataset_name, "Validated and kept only valid team_id entries.")

    return df

# Function to clean and validate all datasets
def clean_and_validate_all(dataframes):
    for dataset_name, df in dataframes.items():
        print(f"Cleaning and validating {dataset_name}...")
        dataframes[dataset_name] = clean_and_validate_dataset(df, dataset_name, dataframes)
    return dataframes

# Run the cleaning and validation process
dataframes = clean_and_validate_all(dataframes)

# Display change logs
print("Change Logs:")
for dataset, logs in change_logs.items():
    print(f"Dataset: {dataset}")
    for log in logs:
        print(f"  - {log}")



Cleaning and validating mls/players...
Cleaning and validating mls/players/xgoals...
Cleaning and validating mls/players/xpass...
Cleaning and validating mls/players/salaries...
Cleaning and validating mls/teams...
Cleaning and validating mls/games...
Change Logs:
Dataset: mls/players
  - 2024-12-05 15:50:45.155690 - Filled missing birth_date with '1900-01-01'.
  - 2024-12-05 15:50:45.155690 - Filled missing values in primary_broad_position with 'UNKNOWN'.
  - 2024-12-05 15:50:45.155690 - Filled missing values in primary_general_position with 'UNKNOWN'.
Dataset: mls/players/xgoals
  - 2024-12-05 15:50:45.163005 - Dropped 263 rows with unhashable types.
  - 2024-12-05 15:50:45.175387 - Validated and kept only valid player_id entries.
Dataset: mls/players/xpass
  - 2024-12-05 15:50:45.175722 - Dropped 263 rows with unhashable types.
  - 2024-12-05 15:50:45.175722 - Validated and kept only valid player_id entries.
Dataset: mls/players/salaries
  - 2024-12-05 15:50:45.175722 - Dropped 11 r

C:\Users\denos\AppData\Local\Temp\ipykernel_21140\3864573399.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  problematic_rows = df.applymap(lambda x: isinstance(x, (dict, list))).any(axis=1)
C:\Users\denos\AppData\Local\Temp\ipykernel_21140\3864573399.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  problematic_rows = df.applymap(lambda x: isinstance(x, (dict, list))).any(axis=1)
C:\Users\denos\AppData\Local\Temp\ipykernel_21140\3864573399.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: str(x) if isinstance(x, (str, int, float)) else None)
C:\Users\denos\AppData\Local\Temp\ipykernel_21140\3864573399.py:55: SettingWithCopyWar

In [70]:
import pyodbc
import pandas as pd

# Function to dynamically map Python data types to SQL data types
def get_sql_data_type(dtype, column_name=None):
    if column_name == "player_id":
        return "VARCHAR(50)"  # Ensure player_id is a fixed-length string for indexing
    elif column_name == "primary_broad_position" or column_name == "primary_general_position":
        return "VARCHAR(50)"  # Make sure positions are fixed length for indexing
    elif column_name == "birth_date":
        return "DATE"
    elif column_name == "date_time_utc":
        return "DATETIME"
    elif column_name in ["base_salary", "guaranteed_compensation"]:
        return "DECIMAL(12, 2)"
    elif pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_object_dtype(dtype):
        return "NVARCHAR(255)"  # Limit size for object (string) columns to make indexing possible
    else:
        return "NVARCHAR(255)"  # Default to NVARCHAR(255)

# Main function to upload DataFrame to SQL Server
def upload_to_sql_server(df, table_name, server, database, batch_size=5000):
    conn = pyodbc.connect(f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()

    # Drop table if it exists
    cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}")
    conn.commit()

    # Create table
    create_query = f"CREATE TABLE {table_name} ({', '.join([f'[{col}] {get_sql_data_type(df[col].dtype, col)}' for col in df.columns])})"
    cursor.execute(create_query)
    conn.commit()

    # Insert data in batches
    placeholders = ", ".join(["?" for _ in df.columns])
    insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    for start in range(0, len(df), batch_size):
        batch = df.iloc[start:start + batch_size]
        print(f"Uploading batch {start} to {start + len(batch) - 1}...")
        try:
            cursor.executemany(insert_query, batch.values.tolist())
            conn.commit()
        except Exception as e:
            print(f"Error uploading batch {start} to {start + len(batch) - 1}: {e}")
            problematic_rows = batch
            print(problematic_rows.head())
            conn.close()
            return

    # Create Clustered Index on player_id in mls/players
    if "player_id" in df.columns:
        cursor.execute(f"CREATE CLUSTERED INDEX idx_player_id ON {table_name} (player_id)")
        conn.commit()

    # Create Non-Clustered Indexes based on column names for each dataset
    if table_name == "AU_players":
        # Only index columns that are valid
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_primary_broad_position ON {table_name} (primary_broad_position)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_primary_general_position ON {table_name} (primary_general_position)")
        conn.commit()

    if table_name == "AU_players_xgoals":
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_team_id_xgoals ON {table_name} (team_id)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_goals_xgoals ON {table_name} (goals)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_xgoals_plus_xassists ON {table_name} (xgoals_plus_xassists)")
        conn.commit()

    if table_name == "AU_players_xpass":
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_team_id_xpass ON {table_name} (team_id)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_attempted_passes_xpass ON {table_name} (attempted_passes)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_pass_completion_percentage_xpass ON {table_name} (pass_completion_percentage)")
        conn.commit()

    if table_name == "AU_players_salaries":
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_team_season_salary ON {table_name} (team_id, season_name_initial)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_base_salary ON {table_name} (base_salary)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_guaranteed_compensation ON {table_name} (guaranteed_compensation)")
        conn.commit()

    if table_name == "AU_teams":
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_team_name ON {table_name} (team_name)")
        conn.commit()

    if table_name == "AU_games":
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_home_team_id ON {table_name} (home_team_id)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_away_team_id ON {table_name} (away_team_id)")
        cursor.execute(f"CREATE NONCLUSTERED INDEX idx_date_time_utc ON {table_name} (date_time_utc)")
        conn.commit()

    # Close the connection
    conn.close()
    print(f"Data successfully uploaded and indexed in {table_name}")

# Workflow to process and upload each DataFrame
server_name = "RAMSEY_BOLTON\\SQLEXPRESS"
database_name = "SoccerProjects"

# Process and upload data
for table_name, df in dataframes.items():
    sql_table_name = table_name.replace("mls/", "AU_").replace("/", "_")  # Customize table naming
    print(f"Uploading {sql_table_name}...")
    upload_to_sql_server(df, sql_table_name, server_name, database_name, batch_size=5000)



Uploading AU_players...
Uploading batch 0 to 999...
Data successfully uploaded and indexed in AU_players
Uploading AU_players_xgoals...
Uploading batch 0 to 611...
Data successfully uploaded and indexed in AU_players_xgoals
Uploading AU_players_xpass...
Uploading batch 0 to 611...
Data successfully uploaded and indexed in AU_players_xpass
Uploading AU_players_salaries...
Uploading batch 0 to 583...
Data successfully uploaded and indexed in AU_players_salaries
Uploading AU_teams...
Uploading batch 0 to 29...
Data successfully uploaded and indexed in AU_teams
Uploading AU_games...
Uploading batch 0 to 999...
Data successfully uploaded and indexed in AU_games
